In [13]:
import pandas as pd
import numpy as np
import os
import json
import altair as alt
import numpy as np

In [48]:
JSON_FILE = "../results/TP53/TP53_codons.fasta.BUSTEDS.json"
pvalueThreshold = 0.05

In [15]:
def getBUSTEDS_ER(json_file):
    with open(json_file, "r") as in_d:
        json_data = json.load(in_d)
    return json_data["Evidence Ratios"]
#end method


def getBUSTEDS_TestResults(json_file):
    with open(json_file, "r") as in_d:
        json_data = json.load(in_d)
    return json_data["test results"]
#end method

def getBUSTEDS_Fits(json_file):
    with open(json_file, "r") as in_d:
        json_data = json.load(in_d)
    return json_data["fits"]
#end method

## ER Plots


In [38]:
data      = getBUSTEDS_ER(JSON_FILE)
num_sites =  len(data["constrained"][0])
res = [x+1 for x in range(num_sites)]
data_tuples = list(zip(res,data["constrained"][0]))
df = pd.DataFrame(data_tuples, columns=['Site','ER'])
df["2ln(ER)"] = 2 * np.log(df["ER"])
df

,Site,ER,2ln(ER)
0,1,1.187809,3.442207e-01
1,2,1.008919,1.775975e-02
2,3,1.064080,1.242207e-01
3,4,0.935883,-1.325297e-01
4,5,1.066060,1.279385e-01
...,...,...,...
1162,1163,1.000000,-5.329071e-15
1163,1164,1.109066,2.070373e-01
1164,1165,1.047390,9.260309e-02
1165,1166,1.101828,1.939419e-01


In [39]:
#source = df[df["omega"] < 10]
source = df

line = alt.Chart(source).mark_line().encode(
    x='Site',
    y='2ln(ER)', 
).properties(
    width=800,
    height=600)

line

alt.Chart(...)

## Fits plots

In [62]:
# Unconstrained model

data   = getBUSTEDS_Fits(JSON_FILE)
df_Unc = pd.DataFrame.from_dict(data["Unconstrained model"]["Rate Distributions"]["Test"], orient='index')

source = df_Unc

alt.Chart(source).mark_circle().encode(
    x='sum(proportion)',
    y='proportion',
    color=alt.Color('omega', scale=alt.Scale(scheme='reds', reverse=False))
).properties(
    width=500,
    height=400)

alt.Chart(...)

In [59]:
# Constrained model
data   = getBUSTEDS_Fits(JSON_FILE)
df_Con = pd.DataFrame.from_dict(data["Constrained model"]["Rate Distributions"]["Test"], orient='index')

source = df_Con

alt.Chart(source).mark_bar().encode(
    x='sum(proportion)',
    y='proportion',
    color=alt.Color('omega', scale=alt.Scale(scheme='reds', reverse=False))
).properties(
    width=500,
    height=400)


alt.Chart(...)

In [53]:
# Figure Legend

result = getBUSTEDS_TestResults(JSON_FILE)
print(result)

result_response = ""

if result["p-value"] <= pvalueThreshold:
    result_response = "found evidence (LRT, p-value ≤ 0.05)"
else:
    result_response = "did NOT find evidence (LRT, p-value > 0.05)"


msg = "BUSTEDS found evidence " + result_response + " of gene-wide episodic diversifying selection in the selected test branches of your phylogeny."
print("\n" + msg)

{'LRT': 385.8230704427115, 'p-value': 0}

BUSTEDS found evidence found evidence (LRT, p-value ≤ 0.05) of gene-wide episodic diversifying selection in the selected test branches of your phylogeny.
